# Convolutional Neural Networks with Keras (3)

This builds on the previous notebook, where 

1) the bottleneck features of VGG16 on our images were recorded and stored

2) a top level classifier was trained on top of those features

##### Here we finally build a unified model and additionally, try fine tuning the last convolutional layer.

Steps:

1) instantiate the convolutional base of VGG16 and load its weights

2) add our previously defined fully-connected model on top, and load its weights

3) create a unified model for fine tuning

3) freeze the layers of the VGG16 model up to the last convolutional block

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.models import Model, load_model
from keras import optimizers
import numpy as np
import os

Using TensorFlow backend.


In [2]:
img_width, img_height = 500,375

train_data_dir = 'data/train'
test_data_dir = 'data/test'

n_train_samples = 235
n_train_ants = 114
n_train_bees = 121

n_test_samples = 148
n_test_ants = 66
n_test_bees = 82

num_epochs = 10
batch_size = 16

top_model_weights_path = 'bottleneck_fc_model.h5'
final_model_weights_path = "final_model.h5"

tuned_model_exists = True

## Part 1: Build a unified model

### Load convolutional layers from VGG16

In [3]:
# load the VGG16 network
base_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_height, img_width,3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 375, 500, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 375, 500, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 375, 500, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 187, 250, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 187, 250, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 187, 250, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 93, 125, 128)      0         
__________

### Add classifier on top, using the saved weights from the training before

In [4]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [5]:
# we don't need the numpy arrays any more, as we have already stored the weights of the top model
top_model.load_weights(top_model_weights_path)
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 84480)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               21627136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 21,627,393
Trainable params: 21,627,393
Non-trainable params: 0
_________________________________________________________________


### Build unified model

In [6]:
# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 375, 500, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 375, 500, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 375, 500, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 187, 250, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 187, 250, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 187, 250, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 93, 125, 128)      0         
__________

## Part 2: Fine tune the overall model

### Freeze all layers before last conv block

In [7]:
# set the first 15 layers (up to the last conv block) to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [8]:
# compile the model with a SGD/momentum optimizer and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

### Fine tune model using very small learning rate

In [9]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


Found 235 images belonging to 2 classes.
Found 148 images belonging to 2 classes.


In [11]:
if not tuned_model_exists:
    # fine-tune the model
    model.fit_generator(
        train_generator,
        steps_per_epoch=n_train_samples // batch_size,
        epochs=num_epochs,
        validation_data=test_generator,
        validation_steps=n_test_samples // batch_size,
        verbose=2)


In [12]:
if not tuned_model_exists:
    model.save(final_model_weights_path)

### Test the model

In [13]:
if tuned_model_exists:
    model = load_model(final_model_weights_path)
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 375, 500, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 375, 500, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 375, 500, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 187, 250, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 187, 250, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 187, 250, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 93, 125, 128)      0         
__________

In [14]:
test_img_path = 'data/train/bees/1092977343_cb42b38d62.jpg'
img = load_img(test_img_path) 
candidate = np.expand_dims(img_to_array(img)/255, axis=0)
model.predict(candidate)

array([[ 1.]], dtype=float32)

In [15]:
def score_img(img_path):
    img = load_img(img_path) 
    candidate = np.expand_dims(img_to_array(img)/255, axis=0)
    pred = model.predict(candidate, verbose=0)
    pred_class = (pred > 0.5).astype('int32')
    #print("class: {}    prob: {}    pic: {}".format(pred_class, pred, img_path))
    return pred_class

In [16]:
num_ants_found = 0
ants_dir = os.path.join(test_data_dir, "ants")
for imgfile in os.listdir(ants_dir):
    score = score_img(os.path.join(ants_dir, imgfile))
    if score == 0: num_ants_found = num_ants_found + 1
print('Detected ant in {} of {} images'.format(num_ants_found, n_test_ants))

Detected ant in 59 of 66 images


In [17]:
num_bees_found = 0
bees_dir = os.path.join(test_data_dir, "bees")
for imgfile in os.listdir(bees_dir):
    score = score_img(os.path.join(bees_dir, imgfile))
    if score == 1: num_bees_found = num_bees_found + 1
print('Detected bee in {} of {} images'.format(num_bees_found, n_test_bees))

Detected bee in 73 of 82 images


In [18]:
# let's say ants are positive
true_positives = num_ants_found
false_negatives = n_test_ants - true_positives
true_negatives = num_bees_found
false_positives = n_test_bees - true_negatives

accuracy = (true_positives + true_negatives) / (n_test_ants + n_test_bees)
sensitivity = true_positives / (true_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)

print('Accuracy: {:.3f}, sensitivity: {:.3f}, precision: {:.3f}'.format(accuracy, sensitivity, precision))

Accuracy: 0.892, sensitivity: 0.894, precision: 0.868
